# GRPC Inference

## Setup

Verify that following variable settings match your deployed model's resource name and grpc URL. The following code assumes that the kube service is in the same namespace, but you could refer to it in full with the namespace, for example: `http://modelmesh-serving.project-name.svc.cluster.local:8008/v2/models/fraud/infer`

In [ ]:
# These modules are already installed on Standard Data Science image
# !pip install grpcio grpcio-tools

In [ ]:
grpc_host = 'modelmesh-serving.ai-example-multi-model-serving.svc.cluster.local'
grpc_port = 8033
model_name = 'fraud-detection-model'

## Inspect the gRPC Endpoint

Check the gRPC endpoint's model metadata:

In [ ]:
import sys
sys.path.append('./utils')

# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import utils.grpc_predict_v2_pb2 as grpc_predict_v2_pb2
import utils.grpc_predict_v2_pb2_grpc as grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

### Request Function

Build and submit the gRPC request. 

Note: You submit the data in the same format that you used for an ONNX inference.


In [ ]:
import numpy as np

def grpc_request(data):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "dense_input"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 5])
    inputs[0].contents.fp32_contents.extend(data)
    print(inputs)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    response = stub.ModelInfer(request)
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    return result_arr

### Run the Request

In [ ]:
data = [0.3111400080477545, 1.9459399775518593, 1.0, 0.0, 0.0]
prediction = grpc_request(data)

In [ ]:
prediction[0]

In [ ]:
threshhold = 0.995

if (prediction > threshhold):
    print('fraud')
else:
    print('not fraud')